In [12]:
import os
import torch as th
import numpy as np
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import matplotlib.pyplot as plt
from scripts.utils import *
from scripts.visualizations import *
from config import *
from datetime import datetime, timedelta
from environment import AircraftDisruptionEnv
from scripts.visualizations import StatePlotter

# Load the environment and scenario data function
def load_scenario_data(scenario_folder):
    file_keys = ['aircraft', 'airports', 'alt_aircraft', 'alt_airports', 'alt_flights', 'config', 'dist', 'flights', 'itineraries', 'position', 'rotations']
    file_paths = {key: os.path.join(scenario_folder, f"{key}.csv") for key in file_keys}

    data_dict = {}
    file_parsing_functions = {
        'config': FileParsers.parse_config,
        'airports': FileParsers.parse_airports,
        'dist': FileParsers.parse_dist,
        'flights': FileParsers.parse_flights,
        'aircraft': FileParsers.parse_aircraft,
        'rotations': FileParsers.parse_rotations,
        'itineraries': FileParsers.parse_itineraries,
        'position': FileParsers.parse_position,
        'alt_flights': FileParsers.parse_alt_flights,
        'alt_aircraft': FileParsers.parse_alt_aircraft,
        'alt_airports': FileParsers.parse_alt_airports
    }

    for file_type, file_path in file_paths.items():
        file_lines = read_csv_with_comments(file_path)
        if file_lines:
            parse_function = file_parsing_functions.get(file_type)
            if parse_function:
                parsed_data = parse_function(file_lines)
                data_dict[file_type] = parsed_data
            else:
                print(f"No parser available for file type: {file_type}")
        else:
            data_dict[file_type] = None
            print(f"{file_type.capitalize()} file is empty")

    return data_dict

# Load the model and run inference
def run_inference(model_path, scenario_folder):
    # Load the trained model
    model = DQN.load(model_path)

    # Load the scenario data
    data_dict = load_scenario_data(scenario_folder)

    # Extract necessary data for the environment
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # Initialize the environment
    env = AircraftDisruptionEnv(aircraft_dict, flights_dict, rotations_dict, alt_aircraft_dict, config_dict)
    env = DummyVecEnv([lambda: env])

    # Reset the environment for inference
    obs = env.reset()  # Only one value returned

    done = False
    total_reward = 0
    step_num = 0

    # Create StatePlotter object for visualizing the environment state
    state_plotter = StatePlotter(
        aircraft_dict=aircraft_dict, 
        flights_dict=flights_dict, 
        rotations_dict=rotations_dict, 
        alt_aircraft_dict=alt_aircraft_dict,
        start_datetime=env.get_attr('start_datetime')[0],  # Extract from environment
        end_datetime=env.get_attr('end_datetime')[0]       # Extract from environment
    )

    while not done:
        # Visualize the environment at each step
        print(f"Step {step_num}:")
        
        # Extract necessary information from the environment for plotting
        swapped_flights = env.get_attr('swapped_flights')[0]
        environment_delayed_flights = env.get_attr('environment_delayed_flights')[0]
        current_datetime = env.get_attr('current_datetime')[0]

        # Use StatePlotter to plot the current state
        state_plotter.plot_state(flights_dict, swapped_flights, environment_delayed_flights, current_datetime)

        # Use the trained model to predict the action
        action = model.predict(obs, deterministic=True)[0]

        # Step the environment
        obs, reward, done, info = env.step(action)

        # Accumulate the reward
        total_reward += reward

        print(f"Action taken: {action}, Reward: {reward}")

        step_num += 1

    print(f"Total Reward: {total_reward}")

# Specify the model path and the scenario folder you want to infer
MODEL_PATH = "Models/dqn_20240914-1"  # Path to the trained DQN model
SCENARIO_FOLDER = "Data/Example/Busy-scenario"  # Path to the new scenario for inference

# Run inference
run_inference(MODEL_PATH, SCENARIO_FOLDER)


FileNotFoundError: [Errno 2] No such file or directory: 'Model/dqn_20240914-1.zip'